In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import numpy as np
import yaml
from matplotlib import pyplot as plt
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)
import os
os.environ.pop("MPLDEBUG", None)
import tqdm
import torch
from torch.utils.data import DataLoader
from model import TripleStreamsVAE
from data import get_triplestream_dataset
from helpers.control_feature_utils import run_inference_and_extract_features, run_inference, extract_control_features



Could not import fluidsynth. AUDIO rendering will not work.
Holoviews not installed. Please install holoviews to be able to generate heatmaps.


## Load Dataset

### Note: First run, it will take a while to load the dataset, but it will be cached for future runs.

In [3]:
config = yaml.safe_load(open('../helpers/configs/TripleStreams_0.5.yaml', 'r'))
config['dataset_root_path'] = os.path.join("../", config['dataset_root_path'])

is_testing = False

dataset = get_triplestream_dataset(
        config=config,
        subset_tag="validation",
        use_cached=True,
        downsampled_size=2000 if is_testing else None,
        print_logs=False                                #<---  Set to True to print dataset loading logs
    )

## Load Model

In [4]:
from model import load_model
model_path = 'models/step_1542993.pth'
model = load_model(
    model_path=model_path,
    model_class=TripleStreamsVAE,
    params_dict=config,
    is_evaluating=True
)

## Run Inference

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from data.triple_streams.triple_stream_data_utils import create_multitab_from_HVO_Sequences, compile_into_list_of_hvo_seqs
from bokeh.io import save
from IPython.display import HTML, display
import torch
import os

drum_mapping = {
    "Input Groove": [36],
    "Stream 1": [37],
    "Stream 2": [38],
    "Stream 3": [39],
}

@interact_manual(
    sample=widgets.IntSlider(min=0, max=len(dataset)-1, step=1, value=0, description='Sample'),
    param1=widgets.IntSlider(min=0, max=32, step=1, value=0, description='Output/Input Difference'),
    param2=widgets.IntSlider(min=0, max=9, step=1, value=0, description='Output/Input Accent Difference'),
    param3=widgets.IntSlider(min=0, max=9, step=1, value=0, description='Density Stream 1'),
    param4=widgets.IntSlider(min=0, max=9, step=1, value=0, description='Density Stream 2'),
    param5=widgets.IntSlider(min=0, max=9, step=1, value=0, description='Density Stream 3')
)
def generate_function(sample, param1, param2, param3, param4, param5):
    """Generate and save plot as HTML file"""
    print(f"Generating with parameters: Sample={sample}, Controls=[{param1}, {param2}, {param3}, {param4}, {param5}]")

    model.eval()
    with torch.no_grad():
        input_groove = dataset.input_grooves[sample].unsqueeze(0)
        encoding_control1_token = torch.tensor(param1, dtype=torch.long).unsqueeze(0)
        encoding_control2_token = torch.tensor(param2, dtype=torch.long).unsqueeze(0)
        decoding_control1_token = torch.tensor(param3, dtype=torch.long).unsqueeze(0)
        decoding_control2_token = torch.tensor(param4, dtype=torch.long).unsqueeze(0)
        decoding_control3_token = torch.tensor(param5, dtype=torch.long).unsqueeze(0)

        hvo, latent_z = model.predict(
            flat_hvo_groove=input_groove,
            encoding_control1_token=encoding_control1_token,
            encoding_control2_token=encoding_control2_token,
            decoding_control1_token=9 - decoding_control1_token,
            decoding_control2_token=9 - decoding_control2_token,
            decoding_control3_token=9 - decoding_control3_token
        )

    hvo_sequence_list = compile_into_list_of_hvo_seqs(
        input_hvos = input_groove,
        output_hvos = hvo, 
        metadatas = [dataset.metadata[sample]]
        
    )

    # Create the plot
    tabs = create_multitab_from_HVO_Sequences(
        hvos=hvo_sequence_list[0]
    )

    hvo_sequence_list[0]

interactive(children=(IntSlider(value=0, description='Sample', max=395759), IntSlider(value=0, description='Ou…